In [5]:
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex



In [ ]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")
# create the sentence window node parser# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=10,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

# documents = SimpleDirectoryReader(
#     input_dir="data"
# ).load_data()


documents = SimpleDirectoryReader(
    input_dir="data"
).load_data()

nodes = node_parser.get_nodes_from_documents(documents)

sentence_index = VectorStoreIndex(nodes)
sentence_index.storage_context.persist(persist_dir="Manual")
print("\n vectorstore created \n")


In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from llama_index.core.llms import ChatMessage


Settings.llm = Ollama(
    model="llama3",
    request_timeout=30000.0,
    temperature=0,
    json_mode=True
)


messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = Settings.llm.chat(messages)
print(resp)


In [9]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.core import StorageContext, load_index_from_storage

Settings.llm = Ollama(
    model="llama3",
    request_timeout=30000.0,
    temperature=0, 
    verbose= True
)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")


storage_context = StorageContext.from_defaults(persist_dir="Manual")

sentence_index = load_index_from_storage(storage_context)


query_engine = sentence_index.as_query_engine(
    similarity_top_k=3,
    
    context_prompt=(
        "You are a chatbot who should answer the user's query without adding anything from your prior knowledge and to the point "
        "Answer  only from the given context and DO NOT USE YOUR PRIOR KNOWLEDGE IN ANY CASE! Don't say anything that is not in the context."
        "IF the answer isn't from context, just say YOU DON'T KNOW THE ANSWER and DONOT TRY TO Additional Info!"
    ),
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)


/home/hamna/Documents/FYP/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
window_response = query_engine.query("ACS stands for?")
print(window_response.metadata)

{'42e04b69-d10d-4515-b037-2854e74e4359': {'window': 'ACRONYMS\nACS    Additional Chief Secretary \nADPs    Annual Development Programmes \nAGPR   Accountant General Pakistan Revenue \nAJ&K   Azad Jammu & Kashmir \nAJKCDC  Azad Jammu and Kashmir Central Development Committee \nAJKDWP  Azad Jammu and Kashmir Development Working Party \nAP   Appraisal Paper \nAPCC   Annual Plan Coordination Committee \nAS   Additional Secretary \nB&R   Buildings & Roads \nBCR or B/C Ratio Benefit-Cost Ratio \nBoQs   Bill of Quantities \nC&W    Communication & Works \nCCC   Concept Clearance Committee \nCCI   Council of Common Interest \nCDA   Capital Development Authority \nCDM   Clean Development Mechanism  \nCDWP   Central Development Working Party \nCF&AO  Chief Finance and Accounts Officer \nCGA   Controller General of Accounts \nCPEC   China-Pakistan Economic Corridor \nCPM   Critical Path Method \nCSR   Composite Schedule of Rates \nDC   Deputy Commissioner  \nDC PC   Deputy Chairman Planning Commis

In [11]:
print(window_response)

ACS stands for Additional Chief Secretary.


In [5]:
from pprint import pprint

pprint(window_response.get_formatted_sources)


<bound method Response.get_formatted_sources of Response(response='Based on the provided context, ACS stands for "Assistant Chief Secretary" or "Accounting and Control System".', source_nodes=[NodeWithScore(node=TextNode(id_='42e04b69-d10d-4515-b037-2854e74e4359', embedding=None, metadata={'window': 'ACRONYMS\nACS    Additional Chief Secretary \nADPs    Annual Development Programmes \nAGPR   Accountant General Pakistan Revenue \nAJ&K   Azad Jammu & Kashmir \nAJKCDC  Azad Jammu and Kashmir Central Development Committee \nAJKDWP  Azad Jammu and Kashmir Development Working Party \nAP   Appraisal Paper \nAPCC   Annual Plan Coordination Committee \nAS   Additional Secretary \nB&R   Buildings & Roads \nBCR or B/C Ratio Benefit-Cost Ratio \nBoQs   Bill of Quantities \nC&W    Communication & Works \nCCC   Concept Clearance Committee \nCCI   Council of Common Interest \nCDA   Capital Development Authority \nCDM   Clean Development Mechanism  \nCDWP   Central Development Working Party \nCF&AO  C

In [10]:
import time
# Start timing
start_time = time.time()

# Perform the query
window_response = query_engine.query("ECNEC approved the which institutional arrangements for the project monitoring. (PPP (Amendment) Act 2021)")

# End timing
end_time = time.time()

# Calculate time taken
time_taken = end_time - start_time

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-small-en")  # Replace with your specific tokenizer if needed

# Tokenize the response and count the number of tokens
response_text = str(window_response)
response_tokens = tokenizer.tokenize(response_text)
num_tokens = len(response_tokens)

# Calculate tokens per second
tokens_per_second = num_tokens / time_taken

print(f"Response: {response_text}")
print(f"Number of tokens: {num_tokens}")
print(f"Time taken (seconds): {time_taken:.2f}")
print(f"Tokens per second: {tokens_per_second:.2f}")

Response: According to the provided context information, ECNEC approved the following institutional arrangements for the project monitoring:

"In its meeting of the 1st of October 20, the ECNEC approved the following institutional arrangements for the project monitoring."

These arrangements are mentioned in Chapter 5, Section 5.03, and include the primary responsibility for the implementation of the project and coordination with relevant stakeholders resting with the PD, as well as the PAO's role in monitoring the performance of ongoing projects and chairing monthly review meetings.
Number of tokens: 103
Time taken (seconds): 4.70
Tokens per second: 21.92


In [11]:
import time
# Start timing
start_time = time.time()

# Perform the query
window_response = query_engine.query("To introduce professional project management in the public sector and to make the projects monitoring and evaluation more effective what has been developed.")

# End timing
end_time = time.time()

# Calculate time taken
time_taken = end_time - start_time

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-small-en")  # Replace with your specific tokenizer if needed

# Tokenize the response and count the number of tokens
response_text = str(window_response)
response_tokens = tokenizer.tokenize(response_text)
num_tokens = len(response_tokens)

# Calculate tokens per second
tokens_per_second = num_tokens / time_taken

print(f"Response: {response_text}")
print(f"Number of tokens: {num_tokens}")
print(f"Time taken (seconds): {time_taken:.2f}")
print(f"Tokens per second: {tokens_per_second:.2f}")

Response: **Rewrite**

To introduce professional project management in the public sector and make projects' monitoring and evaluation more effective, a web-based Project Monitoring and Evaluation System (PMES) has been developed to facilitate line ministries and divisions in planning, tracking progress, monitoring, and identifying corrective actions in a timely manner.
Number of tokens: 65
Time taken (seconds): 7.41
Tokens per second: 8.78


In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-small-en")  # Replace with your specific tokenizer if needed
# Tokenize the response and count the number of tokens
response_text = str(window_response)
response_tokens = tokenizer.tokenize(response_text)
num_tokens = len(response_tokens)

print(f"Response: {response_text}")
print(f"Number of tokens: {num_tokens}")

Response: Based on the provided context, ACS stands for "Assistant Chief Secretary" or "Accounting and Control System".
Number of tokens: 23
